In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
list_places = ['data/kor/attraction_places.csv',
               'data/kor/hotel_places.csv',
               'data/kor/restaurant_places.csv']

In [4]:
list_df = [pd.read_csv(csv,
                       names=['placeId', 'name', 'location', 'class'],
                       encoding='ms949') for csv in list_places]

In [ ]:
top10 = model_attr.wv.most_similar(positive=['320359','324887'], negative=['1958940'])
for place in top10:
    print(df_places[df_places['placeId'] == int(place[0])].name)

In [ ]:
#유의미한 태그만 필터하는 패턴
#pattern = re.compile('MM|NNG|VA[+].*|VV[+].*|XR')